# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [21]:
# TODO: Create a .env file with the following variables
# OPENAI_API_KEY="YOUR_KEY"
# CHROMA_OPENAI_API_KEY="YOUR_KEY"
# TAVILY_API_KEY="YOUR_KEY"

In [22]:
# TODO: Load environment variables
load_dotenv()

True

In [23]:
# Verify that env var is available for URL 
os.getenv("OPENAI_BASE_URL")

'https://openai.vocareum.com/v1'

In [24]:
# Load open AI API key 
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### VectorDB Instance

In [25]:
# TODO: Instantiate your ChromaDB Client
chroma_client = chromadb.Client()

In [26]:
# Choose any path you want
# Save VectorDB to a local filepath 
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [27]:
# TODO: Pick one embedding function
# If picking something different than openai, 
# make sure you use the same when loading it
# embedding_fn = embedding_functions.OpenAIEmbeddingFunction()
embeddings_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY
)

In [28]:
# TODO: Create a collection
# Choose any name you want
# collection = chroma_client.create_collection(
#    name="udaplay",
#    embedding_function=embedding_fn
#)
collection = chroma_client.get_or_create_collection(
    "udaplay_games_collection",
    embedding_function=embeddings_fn
)

### Add documents

In [29]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"
file_counter = 0 
for file_name in sorted(os.listdir(data_dir)):

    # Skip non-json files 
    if not file_name.endswith(".json"):
        continue

    # Process json files 
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

    file_counter += 1 
    print(f"Processed JSON file #{file_counter}")

Processed JSON file #1
Processed JSON file #2
Processed JSON file #3
Processed JSON file #4
Processed JSON file #5
Processed JSON file #6
Processed JSON file #7
Processed JSON file #8
Processed JSON file #9
Processed JSON file #10
Processed JSON file #11
Processed JSON file #12
Processed JSON file #13
Processed JSON file #14
Processed JSON file #15


# Query the Vector DB

In [30]:
collection.query(
    query_texts=["halo"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['015', '014']],
 'embeddings': None,
 'documents': [["[Xbox Series X|S] Halo Infinite (2021) - The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.",
   '[Xbox One] Minecraft (2014) - A sandbox game that allows players to build and explore infinite worlds, fostering creativity and adventure.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Name': 'Halo Infinite',
    'Description': "The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting.",
    'Publisher': 'Xbox Game Studios',
    'Genre': 'First-person shooter',
    'YearOfRelease': 2021,
    'Platform': 'Xbox Series X|S'},
   {'Description': 'A sandbox game that allows players to build and explore infinite worlds, fostering creativity and adventure.',
    'Genre': 'Sandbox, Survival',
    'YearOfRelease': 2014,
    'Platform': 'Xbox One',
    'Name': 'Minecraft',
 

In [31]:
collection.query(
    query_texts=["Racing"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['003', '001']],
 'embeddings': None,
 'documents': [['[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
   '[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Name': 'Gran Turismo 5',
    'Publisher': 'Sony Computer Entertainment',
    'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
    'YearOfRelease': 2010,
    'Platform': 'PlayStation 3',
    'Genre': 'Racing'},
   {'Platform': 'PlayStation 1',
    'Publisher': 'Sony Computer Entertainment',
    'Genre': 'Racing',
    'YearOfRelease': 1997,
    'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, se

In [32]:
collection.query(
    query_texts=["Pets"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['003', '006']],
 'embeddings': None,
 'documents': [['[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
   '[Game Boy Color] Pokémon Gold and Silver (1999) - Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Publisher': 'Sony Computer Entertainment',
    'Name': 'Gran Turismo 5',
    'Genre': 'Racing',
    'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
    'Platform': 'PlayStation 3',
    'YearOfRelease': 2010},
   {'Platform': 'Game Boy Color',
    'Description': 'Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.',
    'Genre': 'Role-playing',
    'YearOfRelease': 1999,
    'Name': 'Pokémon Gold and S

Interesting to note that for the term `Pets`, the game Gran Turismo has a higher similarity than the game Pokemon. 

Pokemon emerged 2nd. 

In [33]:
collection.query(
    query_texts=["crime"],
    n_results=2,
    include=['metadatas', 'documents', 'distances']
)

{'ids': [['002', '003']],
 'embeddings': None,
 'documents': [["[PlayStation 2] Grand Theft Auto: San Andreas (2004) - An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.",
   '[PlayStation 3] Gran Turismo 5 (2010) - A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'Platform': 'PlayStation 2',
    'Genre': 'Action-adventure',
    'YearOfRelease': 2004,
    'Description': "An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.",
    'Name': 'Grand Theft Auto: San Andreas',
    'Publisher': 'Rockstar Games'},
   {'Genre': 'Racing',
    'Description': 'A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.',
    'Publisher': 'Sony Computer E